In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn import metrics as mt 
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.models import Model

df = pd.read_csv('Video_games_esrb_rating.csv')

# map the esrb string values to integers for our target
rating = []

for i in df['esrb_rating']:
    if(i == 'E'):
        rating.append(0)
    elif(i == 'ET'):
        rating.append(1)
    elif(i == 'T'):
        rating.append(2)
    else:
        rating.append(3)
        
del df['title'] #get rid of unnecessary data
del df['console']
del df['esrb_rating'] #get rid of original column since its been mapped now

from sklearn.model_selection import ShuffleSplit

y = np.array(rating).T

#remaining columns in dataset are used for X
X = df.to_numpy()

#set up shuffle split using example
num_cv_iterations = 10
num_instances = len(y)
cv_object = ShuffleSplit(n_splits = num_cv_iterations, test_size = 0.2)